In [1]:
import psycopg2

API_KEY = '2CDC7D3628A900A6E59E9BC19A8E0B62'

try:
    # Conexão com o banco de dados
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  # Configuração da codificação
    )

    # Criação de um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Exemplo de execução de uma consulta
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    # Tratamento de erro com codificação segura
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


Versão do PostgreSQL: ('PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.41.34123, 64-bit',)


In [2]:
class GameCategory:
    def __init__(self, id=None, description=None, appid=None):
        self.id = id
        self.description = description
        self.appid = appid

    def __repr__(self):
        return f"GameCategory(id={self.id}, description='{self.description}', appid={self.appid})"

class Category:
    def __init__(self, category_id: int, descricao: str):
        self.category_id = category_id
        self.descricao = descricao

    def __repr__(self):
        return f"Category(category_id={self.category_id}, descricao='{self.descricao}')"


class GameInfo:
    def __init__(
        self, 
        id=None, 
        type=None, 
        name=None, 
        steam_appid=None, 
        required_age=None, 
        is_free=None, 
        supported_languages=None, 
        windows=None, 
        mac=None, 
        linux=None, 
        recommendations=None, 
        release_date=None, 
        coming_soon=None,
        price_overview=None,
    ):
        self.id = id
        self.type = type
        self.name = name
        self.steam_appid = steam_appid
        self.required_age = required_age
        self.is_free = is_free
        self.supported_languages = supported_languages
        self.windows = windows
        self.mac = mac
        self.linux = linux
        self.recommendations = recommendations
        self.release_date = release_date
        self.coming_soon = coming_soon
        self.price_overview = price_overview
    
    def __repr__(self):
        return (
            f"GameInfo(id={self.id}, type={self.type}, name={self.name}, "
            f"steam_appid={self.steam_appid}, required_age={self.required_age}, "
            f"is_free={self.is_free}, supported_languages={self.supported_languages}, "
            f"windows={self.windows}, mac={self.mac}, linux={self.linux}, "
            f"recommendations={self.recommendations}, release_date={self.release_date}, "
            f"coming_soon={self.coming_soon})"
        )
class Game:
    def __init__(self, rank=None, appid=None, last_week_rank=None, peak_in_game=None):
        self.rank = rank
        self.appid = appid
        self.last_week_rank = last_week_rank
        self.peak_in_game = peak_in_game
    
    def __repr__(self):
        return (
            f"Game(rank={self.rank}, appid={self.appid}, last_week_rank={self.last_week_rank}, "
            f"peak_in_game={self.peak_in_game})"
        )


In [5]:
import requests
import json
from datetime import datetime
from currency_converter import CurrencyConverter


URL_TOP_GAMES = 'https://api.steampowered.com/ISteamChartsService/GetMostPlayedGames/v1/'
URL_GAME_DETAILS = 'https://store.steampowered.com/api/appdetails/'

def get_most_played_games(max_games=100):
    params = {
        'key': API_KEY
    }
    try:
        response = requests.get(URL_TOP_GAMES, params=params)
        if response.status_code == 200:
            data = response.json()
            Game = data['response']['ranks'][:max_games]
            # print(Game)
            game_ids = []
        
            for game in Game:
                # print("pico de jogadores: " + game['peak_in_game'])
                # print(f"/inseriu: {game['appid']} ")
                game_id = int(game['appid'])
                rank = int(game['rank'])
                last_week_rank = int(game['last_week_rank'])
                peak_in_game = int(game['peak_in_game'])
                
                updateSql = """
                INSERT INTO steam.games
                ("rank", appid, last_week_rank, peak_in_game)
                VALUES (%s, %s, %s, %s)ON CONFLICT (appid) DO NOTHING
                """
               
                values = (int(game['rank']),int(game['appid']),  int(game['last_week_rank']), int(game['peak_in_game']),) 
                # print(values)
                cursor.execute(updateSql, values)
                cursor.connection.commit()
                    
                game_ids.append(game_id)
        
            return game_ids  # Retorna a lista de game_ids
        
            # print(f"IDs de {len(game_ids)} jogos salvos em 'game_ids.json'")
        
        else:
            print(f"Erro ao obter dados: {response.status_code}")
    except Exception as e:
        cursor.connection.rollback()
        print("Erro ao inserir registro:", e)

def get_game_details(game_id):
    params = {
        'appids': game_id
    }
    response = requests.get(URL_GAME_DETAILS, params=params)

    if response.status_code == 200:
        data = response.json()
        try:
            # Verifique se a chave 'data' está presente para o game_id
            if str(game_id) in data and 'data' in data[str(game_id)]:
                GameInfo = data[str(game_id)]['data']
                # print(GameInfo)  # Apenas para debug
                infoGame = int(GameInfo.get('steam_appid', 0))
                # print(infoGame)
                # return
                prices = GameInfo.get('price_overview', [])
                
                if prices != []:
                    c = CurrencyConverter()
                    from_currency = str(prices['currency'])
                    to_currency = 'USD'
                    if from_currency != 'USD':
                        print(f'Currency {from_currency}')
                        convertidoinit = "{:.2f}".format(int(prices['initial']) / 100) 
                        convertidofin = "{:.2f}".format(int(prices['final']) / 100) 
                        print(convertidofin)
                        print(convertidoinit)
                        converted_init = c.convert(convertidoinit, from_currency, to_currency)
                        converted_fin = c.convert(convertidofin,from_currency, to_currency)
                        print(converted_init)
                        print(converted_fin)
                    else:
                        converted_init = "{:.2f}".format(int(prices['initial']) / 100)
                        converted_fin = "{:.2f}".format(int(prices['final']) / 100) 

                    insertprc = """ insert into steam.game_prices (currency, initial, final,appid)
                    Values(%s, %s, %s, %s)  ON CONFLICT (appid) DO NOTHING"""
                    prcVal = ( 'USD',converted_init,converted_fin, infoGame,)
                    cursor.execute(insertprc, prcVal)
                    cursor.connection.commit()  

                categories_list = GameInfo.get('categories', [])           
                for category in categories_list:
                  
                    insertCate = """ insert into steam.games_categories_steam (id, description, appid)
                    Values(%s, %s, %s)  ON CONFLICT (id,appid) DO NOTHING"""
                    catVal = ( int(category['id']),str(category['description']), infoGame,)
                    cursor.execute(insertCate, catVal)
                    cursor.connection.commit()   


                val = (
                    str(GameInfo.get('type', '')),
                    str(GameInfo.get('name', '')),
                    int(GameInfo.get('steam_appid', 0)),
                    int(GameInfo.get('required_age', 0)),
                    bool(GameInfo.get('is_free', False)),
                    str(GameInfo.get('supported_languages', '')),
                    bool(GameInfo.get('platforms', {}).get('windows', False)),
                    bool(GameInfo.get('platforms', {}).get('mac', False)),
                    bool(GameInfo.get('platforms', {}).get('linux', False)),
                    str(GameInfo.get('release_date', {}).get('date', '')),
                    bool(GameInfo.get('coming_soon', False))
                )

                # Insere os dados no banco de dados
                insert_sql = """  
                    INSERT INTO steam.game_info
                    (type, name, steam_appid, required_age, is_free, supported_languages, windows, mac, linux, release_date, coming_soon)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (steam_appid) DO NOTHING;
                """
                cursor.execute(insert_sql, val)
                cursor.connection.commit()
                print(f"inseriu o jogo {GameInfo.get('name', '')} ")

            else:
                print(f"Erro: Dados não encontrados para o game_id {game_id}")

        except Exception as e:
            cursor.connection.rollback()
            print("Erro ao inserir registro:", e)

    else:
        print(f"Erro ao obter detalhes do jogo {game_id}: {response.status_code}")



if __name__ == "__main__":
    game_ids = get_most_played_games(max_games=100)

    for game_id in game_ids:
        print(f"\nBuscando detalhes para o jogo ID: {game_id}")
        get_game_details(game_id)  


Buscando detalhes para o jogo ID: 730
inseriu o jogo Counter-Strike 2 

Buscando detalhes para o jogo ID: 570
inseriu o jogo Dota 2 

Buscando detalhes para o jogo ID: 578080
inseriu o jogo PUBG: BATTLEGROUNDS 

Buscando detalhes para o jogo ID: 431960
Currency UAH
89.00
89.00
Erro ao inserir registro: UAH is not a supported currency

Buscando detalhes para o jogo ID: 1938090
inseriu o jogo Call of Duty® 

Buscando detalhes para o jogo ID: 1203220
inseriu o jogo NARAKA: BLADEPOINT 

Buscando detalhes para o jogo ID: 271590
inseriu o jogo Grand Theft Auto V 

Buscando detalhes para o jogo ID: 1172470
inseriu o jogo Apex Legends™ 

Buscando detalhes para o jogo ID: 236390
inseriu o jogo War Thunder 

Buscando detalhes para o jogo ID: 2923300
inseriu o jogo Banana 

Buscando detalhes para o jogo ID: 413150
inseriu o jogo Stardew Valley 

Buscando detalhes para o jogo ID: 1643320
inseriu o jogo S.T.A.L.K.E.R. 2: Heart of Chornobyl 

Buscando detalhes para o jogo ID: 252490
Currency SGD
34

In [25]:
import requests
from bs4 import BeautifulSoup
# URL da p�gina de tags de jogos da Steam
URL_TAGS_PAGE = 'https://store.steampowered.com/tag/browse/?l=portuguese'

def get_all_steam_categories():
    # Faz a requisi��o para a p�gina de tags
    response = requests.get(URL_TAGS_PAGE)
    
    # Verifica se a requisi��o foi bem-sucedida
    if response.status_code == 200:
        # Usa BeautifulSoup para analisar o conte�do HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Localiza todas as tags de categoria (ajuste o seletor conforme necess�rio)
        # Nesse caso, estamos assumindo que as categorias est�o na classe 'tag_browse_tag'
        tags = soup.find_all("div", {"class": "tag_browse_tag"})
        
        # Lista para armazenar categorias
        categories = [tag.get_text(strip=True) for tag in tags]
        
        print("Categorias encontradas:")
        for category in categories:
            print(category)
            sql = "INSERT INTO steam.categories (descricao) VALUES (%s) ON CONFLICT (descricao) DO NOTHING; "
            try:
                cursor.execute(sql, (category,))  # Colocando category dentro de uma tupla
                cursor.connection.commit()
            except Exception as e:
                cursor.connection.rollback()
                print("Erro ao inserir registro:", e)
        return categories
    else:
        print(f"Erro ao acessar a p�gina de tags da Steam: {response.status_code}")
        return []

# Executa a fun��o
all_categories = get_all_steam_categories()

# Fechar a conex�o ao banco de dados depois de finalizar



Categorias encontradas:
Indie
Ação
Aventura
Casual
RPG
Um Só Jogador
Simulação
Estratégia
Acesso Antecipado
Grátis para Jogar
2D
3D
Boa Atmosfera
Colorido
História Excelente
Fantasia
Exploração
Multijogador
Quebra-Cabeças
Pixels
Fofo
Combate
Primeira Pessoa
Multijogador em Massa
Desporto
Ação e Aventura
Violento
Engraçado
Arcade
Anime
Relaxante
Ficção Científica
Terror
Jogo de Tiros
Corridas
Comando
Conteúdo Sexual
Estilizado
Terceira Pessoa
Nudez
Para Toda a Família
Retro
Protagonista Feminina
PvE
Co-op
Mundo Aberto
Escolhas Importantes
Violência Explícita
Visto de Cima
PvP
Realista
Sobrevivência
Plataformas
Sinistro
Linear
Comédia
Mistério
Personalização de Personagens
Estilo Cartoon
Romance Visual
Vários Finais
Física
Plataformas 2D
Co-op Online
Terror Psicológico
Tiros em Primeira Pessoa
Magia
Old School
Sandbox
Tático
RPG de Ação
Medieval
Roguelike
Difícil
Futurista
Desenhado à Mão
Minimalista
Construção
Roguelite
Point & Click
Crafting
Gestão
Espaço
Realidade Virtual
Drama
Desenh